## AI Heart Specialist Using DEEP Seek LLM Model

# Develped By: Fida Muhammad Khan

In [24]:
# Step1: Create & Setup hugging face API token in Collab!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-vh5ertbw
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-vh5ertbw
  Resolved https://github.com/unslothai/unsloth.git to commit 6c234d5a66adb76b9b93fb0f2445648199d88e66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192661 sha256=7c95b5b79a9ccccd8861e836f9499aea3191e0d2abd4087bae3802466cd70bc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-b3_5nfo9/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.3.19
    Uninstalling unsloth-2025.3.19:
      Successfully uninstalled unsloth-2025.3.19


In [25]:
# Step3: Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

In [26]:
# Step4: Check HF token
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [27]:
# Optional: Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla T4


In [29]:
pip install transformers bitsandbytes

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "deepseek-ai/deepseek-llm-7b-chat"

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
# Step6: Setup system prompt for Heart Specialist
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a heart specialist specializing in cardiovascular diagnostics, treatment planning, and clinical reasoning. Answer the heart-related medical question below using your advanced knowledge.

### Query:
{}

### Answer:
<think>{}
"""

In [32]:

# Step7: Run Inference on the model

# Define a test question
question = """A 75-year-old man with a long history of chest pain, shortness of breath, and fatigue. She has a history of hypertension, and an ECG shows signs of left ventricular hypertrophy. What diagnostic tests would be most appropriate to further evaluate her condition and potential heart failure?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1500,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response)

["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a heart specialist specializing in cardiovascular diagnostics, treatment planning, and clinical reasoning. Answer the heart-related medical question below using your advanced knowledge.\n\n### Query:\nA 75-year-old man with a long history of chest pain, shortness of breath, and fatigue. She has a history of hypertension, and an ECG shows signs of left ventricular hypertrophy. What diagnostic tests would be most appropriate to further evaluate her condition and potential heart failure?\n\n### Answer:\n<think>\n\n1. Start by considering the patient's symptoms and history, which indicate a 

In [33]:
print(response[0].split("### Answer:")[1])


<think>

1. Start by considering the patient's symptoms and history, which indicate a possible heart failure.
2. The ECG with signs of left ventricular hypertrophy suggests that there might be an underlying heart condition.
3. Since the patient is experiencing symptoms of shortness of breath and fatigue, it is essential to evaluate the overall function and structure of the heart.
4. Based on the patient's history, a comprehensive evaluation of the heart's function and structure is necessary.

### Diagnostic Tests:
1. Cardiac catheterization: This procedure allows for the measurement of cardiac function, including the ejection fraction and blood pressure in different heart chambers. Additionally, it allows for the assessment of coronary artery function, which can help determine the cause of chest pain and shortness of breath.
2. Echocardiography: This non-invasive imaging technique can provide detailed information about the structure and function of the heart, including the size and th